In [1]:
!pip3 install wandb

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import wandb
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()
PROJECT_NAME = 'chatbot'
device = 'cuda'

In [3]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence.lower())

In [4]:
tokenize('$100')

['$', '100']

In [5]:
def stem(word):
    return stemmer.stem(word.lower())

In [6]:
stem('organic')

'organ'

In [7]:
def bag_of_words(tokenized_words,words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(words))
    for idx,w in enumerate(words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [8]:
bag_of_words(['hi'],['hi','how','hi'])

array([1., 0., 1.])

In [9]:
# data = pd.read_csv('./data.csv').sample(frac=1)[:3250]

In [10]:
# data.to_csv('./cleaned-data.csv',index=False)

In [11]:
data = pd.read_json('./data.json')

In [12]:
dict(data.iloc[0]['intents'])

{'tag': 'greeting',
 'patterns': ['Hi',
  'Hey',
  'How are you',
  'Is anyone there?',
  'Hello',
  'Good day'],
 'responses': ['Hey :-)',
  'Hello, thanks for visiting',
  'Hi there, what can I do for you?',
  'Hi there, how can I help?']}

In [13]:
new_data = {'Question':[],'Tag':[]}
responses = {'Tag':[],'Response':[]}

In [14]:
for i in range(len(data)):
    info = data.iloc[i]['intents']
    for q in list(info['patterns']):
        new_data['Tag'].append(info['tag'])
        new_data['Question'].append(q)
    for r in list(info['responses']):
        responses['Tag'].append(info['tag'])
        responses['Response'].append(r)

In [15]:
data = pd.DataFrame(new_data)

In [16]:
data.to_csv('./data.csv')

In [17]:
responses = pd.DataFrame(responses)

In [18]:
X = data['Question']
y = data['Tag']
X_words = []
labels = {}
labels_r = {}
idx = 0
data = []

In [19]:
for label in y:
    if label not in list(labels.keys()):
        idx += 1
        labels[label] = idx
        labels_r[idx] = label

In [20]:
labels

{'greeting': 1,
 'goodbye': 2,
 'thanks': 3,
 'items': 4,
 'payments': 5,
 'delivery': 6,
 'funny': 7}

In [21]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for Xb in X_batch:
        new_X.append(stem(Xb))
    X_words.extend(new_X)
    
    data.append([
        X_words,
        labels[y_batch]-1
    ])

26it [00:00, 9243.25it/s]


In [22]:
X_words = sorted(set(X_words))
np.random.shuffle(data)

In [23]:
X = []
y = []

In [24]:
for input_sentence,output_sentence in tqdm(data):
    X.append(bag_of_words(input_sentence,X_words))
    y.append(output_sentence)

100%|██████████| 26/26 [00:00<00:00, 1072.88it/s]


In [25]:
from sklearn.model_selection import *
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.0125,shuffle=False)
X_train = torch.from_numpy(np.array(X_train)).to(device).long()
y_train = torch.from_numpy(np.array(y_train)).to(device).long()
X_test = torch.from_numpy(np.array(X_test)).to(device).long()
y_test = torch.from_numpy(np.array(y_test)).to(device).long()

In [26]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [27]:
def get_accuracy(model,X,y,):
    preds = model(X)
    correct = 0
    total = 0
    for pred,yb in zip(preds,y):
        pred = int(torch.argmax(pred))
        yb = int(torch.argmax(yb))
        if pred == yb:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [28]:
class Model(Module):
    def __init__(self, hidden_size=8,):
        super().__init__()
        self.l1 = Linear(len(X_words), hidden_size) 
        self.l2 = Linear(hidden_size, hidden_size) 
        self.l3 = Linear(hidden_size, len(labels))
        self.relu = ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [31]:
model = Model().to(device)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 1000
batch_size = 8

In [32]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    preds = model(X_train)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model.eval()
    torch.cuda.empty_cache()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    torch.cuda.empty_cache()
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
    torch.cuda.empty_cache()
    model.train()
wandb.finish()
torch.cuda.empty_cache()

  0%|          | 0/1000 [00:00<?, ?it/s]


RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'target' in call to _thnn_nll_loss_forward

In [ ]:
torch.save(model.to('cpu'),'model.pt')
torch.save(model.to('cpu'),'model.pth')
torch.save(model.to('cpu').state_dict(),'model-sd.pt')
torch.save(model.to('cpu').state_dict(),'model-sd.pth')
torch.save(X_words,'words.pt')
torch.save(X_words,'words.pth')
torch.save(data,'data.pt')
torch.save(data,'data.pth')
torch.save(labels,'labels.pt')
torch.save(labels,'labels.pth')
torch.save(labels_r,'labels_r.pt')
torch.save(labels_r,'labels_r.pth')
torch.save(idx,'idx.pt')
torch.save(idx,'idx.pth')
torch.save(y_train,'y_train.pt')
torch.save(y_test,'y_test.pth')
torch.save(y,'y.pt')
torch.save(y,'y.pth')

In [ ]:
model.eval()
torch.argmax(model(torch.from_numpy(bag_of_words(['hi','how','are','you'],X_words)).float()))

In [ ]:
responses

In [ ]:
responses.to_csv('./responses.csv')